In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np


In [11]:
# Rutas a tus carpetas
data_root = "images"
train_dir = os.path.join(data_root, "seg_train", "seg_train")
test_dir  = os.path.join(data_root, "seg_test",  "seg_test")
pred_dir  = os.path.join(data_root, "seg_pred",  "seg_pred")

print(train_dir)
print(test_dir)
print(pred_dir)

images\seg_train\seg_train
images\seg_test\seg_test
images\seg_pred\seg_pred


In [ ]:
#Transformaciones mínimas: redimensionar + tensor + normalizar
IMG_SIZE = 150

transform_base = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

NameError: name 'transforms' is not defined

In [ ]:
#Transformaciones mínimas: redimensionar + tensor + normalizar
IMG_SIZE = 150

transform_base = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Aumento de datos para entrenamiento
transform_train = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),

    # 1 - Rotación aleatoria
    transforms.RandomRotation(degrees=20),

    # 2 - Flip horizontal aleatorio
    transforms.RandomHorizontalFlip(p=0.5),

    # 3 - Recorte aleatorio (primero aumento tamaño y luego recorte)
    transforms.RandomResizedCrop(size=IMG_SIZE, scale=(0.8, 1.0)),

    # 4 - Cambios aleatorios de brillo/contraste/saturación
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),

    # 5 - Escala de grises parcial (con probabilidad 0.2)
    transforms.RandomApply([transforms.RandomGrayscale(p=1.0)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
batch_size = 4
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(test_dataset,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=2)

pred_loader = DataLoader(pred_dataset,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=2)

In [ ]:
# Dataset auxiliar sólo con resize (para ver originales)
transform_only_resize = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_dataset_no_aug = datasets.ImageFolder(root=train_dir,
                                            transform=transform_only_resize)
train_loader_no_aug = DataLoader(train_dataset_no_aug,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 num_workers=2)

Lote de imagenes originales

In [ ]:
images_orig, labels_orig = next(iter(train_loader_no_aug))

plt.figure(figsize=(8, 8))
grid = utils.make_grid(images_orig, nrow=2)  # 2x2 = 4
imshow_tensor(grid, title="Imágenes originales")
plt.show()

Lote de imagenes transformadas


In [ ]:
#Obtener índices reales de las imágenes en train_dataset
indices = [i for i in range(batch_size)]
paths = [train_dataset.imgs[i][0] for i in indices]
labels = [train_dataset.imgs[i][1] for i in indices]

# Cargar y transformar con aumento
from PIL import Image

images_aug = []
for p in paths:
    img = Image.open(p).convert("RGB")
    img_t = transform_train(img)
    images_aug.append(img_t)
images_aug = torch.stack(images_aug)

plt.figure(figsize=(8, 8))
grid_aug = utils.make_grid(images_aug, nrow=2)
imshow_tensor(grid_aug, title="Imágenes transformadas (aumento)")
plt.show()

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        # Bloque conv 1: entrada 3x150x150 -> salida 32x75x75
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,
                               kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bloque conv 2: entrada 32x75x75 -> salida 64x37x37
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,
                               kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Cálculo de tamaño después de las capas:
        # 150x150 -> pool -> 75x75 -> pool -> 37x37
        conv_out_size = 64 * 37 * 37

        self.fc1 = nn.Linear(conv_out_size, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        x = x.view(x.size(0), -1)  # a vector

        x = self.relu3(self.fc1(x))
        x = self.fc2(x)  # logits para CrossEntropyLoss
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=num_classes).to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()

learning_rate = 1e-4
weight_decay  = 1e-4

optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate,
                       weight_decay=weight_decay)

In [ ]:
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Época [{epoch+1}/{num_epochs}] - Pérdida entrenamiento: {epoch_loss:.4f}")

In [ ]:
model.eval()
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader.dataset)
accuracy = 100.0 * correct / total

print(f"Pérdida en test: {avg_test_loss:.4f}")
print(f"Precisión en test: {accuracy:.2f}%")    